In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

Using TensorFlow backend.


There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [3]:
print(dog_names)

['Affenpinscher', 'Afghan_hound', 'Airedale_terrier', 'Akita', 'Alaskan_malamute', 'American_eskimo_dog', 'American_foxhound', 'American_staffordshire_terrier', 'American_water_spaniel', 'Anatolian_shepherd_dog', 'Australian_cattle_dog', 'Australian_shepherd', 'Australian_terrier', 'Basenji', 'Basset_hound', 'Beagle', 'Bearded_collie', 'Beauceron', 'Bedlington_terrier', 'Belgian_malinois', 'Belgian_sheepdog', 'Belgian_tervuren', 'Bernese_mountain_dog', 'Bichon_frise', 'Black_and_tan_coonhound', 'Black_russian_terrier', 'Bloodhound', 'Bluetick_coonhound', 'Border_collie', 'Border_terrier', 'Borzoi', 'Boston_terrier', 'Bouvier_des_flandres', 'Boxer', 'Boykin_spaniel', 'Briard', 'Brittany', 'Brussels_griffon', 'Bull_terrier', 'Bulldog', 'Bullmastiff', 'Cairn_terrier', 'Canaan_dog', 'Cane_corso', 'Cardigan_welsh_corgi', 'Cavalier_king_charles_spaniel', 'Chesapeake_bay_retriever', 'Chihuahua', 'Chinese_crested', 'Chinese_shar-pei', 'Chow_chow', 'Clumber_spaniel', 'Cocker_spaniel', 'Collie',

In [4]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [7]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors_original = paths_to_tensor(train_files).astype('float32')
valid_tensors_original = paths_to_tensor(valid_files).astype('float32')
test_tensors_original = paths_to_tensor(test_files).astype('float32')

100%|██████████| 836/836 [00:12<00:00, 68.01it/s]


In [8]:
np.save(open('train_tensors_original.npy', 'w'), train_tensors_original)
np.save(open('valid_tensors_original.npy', 'w'), valid_tensors_original)
np.save(open('test_tensors_original.npy', 'w'), test_tensors_original)

In [8]:
# Transfer Learning. RESNET50
from keras.applications.resnet50 import ResNet50
resnet = ResNet50(include_top=False, weights='imagenet')
print "Done extracting features"

Done extracting features


In [13]:
import numpy as np
train_tensors_original = np.load(open('train_tensors_original.npy'))
test_tensors_original = np.load(open('test_tensors_original.npy'))
valid_tensors_original = np.load(open('valid_tensors_original.npy'))

In [14]:
from keras.applications.resnet50 import preprocess_input
train_tensors1 = preprocess_input(train_tensors_original)
valid_tensors1 = preprocess_input(valid_tensors_original)
test_tensors1 = preprocess_input(test_tensors_original)

In [4]:
train_resnet_features = resnet.predict(train_tensors1)
print(train_resnet_features.shape)

(6680, 1, 1, 2048)


In [5]:
valid_resnet_features = resnet.predict(valid_tensors1)
print(valid_resnet_features.shape)

(835, 1, 1, 2048)


In [6]:
test_resnet_features = resnet.predict(test_tensors1)
print(test_resnet_features.shape)

(836, 1, 1, 2048)


In [7]:
np.save(open('bottleneck_features//train_resnet_features.npy', 'w'), train_resnet_features)
np.save(open('bottleneck_features//valid_resnet_features.npy', 'w'), valid_resnet_features)
np.save(open('bottleneck_features//test_resnet_features.npy', 'w'), test_resnet_features)

In [3]:
# Alternatively just use saved pretrained features
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_resnet_features = bottleneck_features['train']
valid_resnet_features = bottleneck_features['valid']
test_resnet_features = bottleneck_features['test']

In [2]:
train_resnet_features = np.load(open('bottleneck_features//train_resnet_features.npy'))
valid_resnet_features = np.load(open('bottleneck_features//valid_resnet_features.npy'))
test_resnet_features = np.load(open('bottleneck_features//test_resnet_features.npy'))

In [32]:
print(train_resnet_features.shape)

(6680, 1, 1, 2048)


In [48]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

#Top model
ResNet_model = Sequential()
ResNet_model.add(GlobalAveragePooling2D(input_shape=train_resnet_features.shape[1:]))
ResNet_model.add(Dropout(0.5))

# ResNet_model.add(Dense(266, activation='relu'))
# ResNet_model.add(Dropout(0.5))

ResNet_model.add(Dense(133, activation='softmax'))

ResNet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ResNet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [49]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.resnet_ft.hdf5', 
                               verbose=1, save_best_only=True)


ResNet_model.fit(train_resnet_features, train_targets, 
          batch_size = 128,
          epochs=30,
          verbose=1,
          validation_data=(valid_resnet_features, valid_targets),
          callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)
          ])

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s 147us/step - loss: 3.3864 - acc: 0.2608 - val_loss: 1.4471 - val_acc: 0.6551
Epoch 2/30
6680/6680 [==============================] - 0s 36us/step - loss: 1.2599 - acc: 0.6579 - val_loss: 0.9393 - val_acc: 0.7365
Epoch 3/30
6680/6680 [==============================] - 0s 37us/step - loss: 0.7963 - acc: 0.7781 - val_loss: 0.7842 - val_acc: 0.7713
Epoch 4/30
6680/6680 [==============================] - 0s 37us/step - loss: 0.5885 - acc: 0.8364 - val_loss: 0.7107 - val_acc: 0.7808
Epoch 5/30
6680/6680 [==============================] - 0s 38us/step - loss: 0.4686 - acc: 0.8710 - val_loss: 0.6783 - val_acc: 0.7868
Epoch 6/30
6680/6680 [==============================] - 0s 43us/step - loss: 0.3824 - acc: 0.8945 - val_loss: 0.6369 - val_acc: 0.8036
Epoch 7/30
6680/6680 [==============================] - 0s 42us/step - loss: 0.3162 - acc: 0.9142 - val_loss: 0.6309 - val_acc: 0.8024
Epoch 8

Epoch 28/30
6680/6680 [==============================] - 0s 34us/step - loss: 0.0599 - acc: 0.9879 - val_loss: 0.5748 - val_acc: 0.8299
Epoch 29/30
6680/6680 [==============================] - 0s 35us/step - loss: 0.0521 - acc: 0.9927 - val_loss: 0.5632 - val_acc: 0.8383
Epoch 30/30
6680/6680 [==============================] - 0s 34us/step - loss: 0.0534 - acc: 0.9901 - val_loss: 0.5669 - val_acc: 0.8287


In [50]:
### Load the model weights with the best validation loss.
ResNet_model.load_weights('saved_models/weights.best.resnet_ft.hdf5')

In [51]:
# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet_model.predict(np.expand_dims(feature, axis=0))) for feature in test_resnet_features]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 82.0000%


In [39]:
from keras.models import Model
base_model = keras.applications.ResNet50(include_top=False, weights='imagenet')
top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=train_resnet_features.shape[1:]))
top_model.add(Dense(133, activation='softmax'))
top_model.load_weights('saved_models/weights.best.resnet_ft.hdf5')
model = Model(inputs = base_model.input, outputs = top_model(base_model.output))

In [52]:
from keras.models import Model
base_model = resnet
top_model = ResNet_model
model = Model(inputs = base_model.input, outputs = top_model(base_model.output))

In [53]:
from keras import optimizers
for layer in model.layers[:169]:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr1e-4, momentum=0.9), metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [31]:
from keras.preprocessing.image import ImageDataGenerator

# create and configure augmented image generator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 20,
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True, # randomly flip images horizontally
    shear_range = 0.2, # rnadomly applying shear transformation
    zoom_range = 0.2, # randomly zooming inside pictures
    fill_mode = 'nearest')
# fit augmented image generator on data
datagen.fit(train_tensors_original)

In [54]:
# WITH DATA AUGMENTATION (need to recompile the model before running)

from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping

epochs = 10
batch_size = 64

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_resnet_augmented.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

## Using Image Augmentation
# model.fit_generator(datagen.flow(train_tensors_original, train_targets, batch_size=batch_size),
#                     validation_data=(valid_tensors1, valid_targets), 
#                     steps_per_epoch=train_tensors_original.shape[0] // batch_size,
#                     epochs=epochs, callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)



model.fit(train_tensors1, train_targets, batch_size=batch_size,
                    validation_data=(valid_tensors1, valid_targets), 
                    epochs=epochs, callbacks=[checkpointer, EarlyStopping(min_delta=1e-7, patience=15)], verbose=1)



Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 119s 18ms/step - loss: 0.3368 - acc: 0.9003 - val_loss: 1.0308 - val_acc: 0.7234
Epoch 2/10
6680/6680 [==============================] - 119s 18ms/step - loss: 0.3298 - acc: 0.8994 - val_loss: 1.0237 - val_acc: 0.7234
Epoch 3/10
6680/6680 [==============================] - 119s 18ms/step - loss: 0.3289 - acc: 0.9009 - val_loss: 1.0196 - val_acc: 0.7210
Epoch 4/10
6680/6680 [==============================] - 118s 18ms/step - loss: 0.3152 - acc: 0.9060 - val_loss: 1.0088 - val_acc: 0.7257
Epoch 5/10
6680/6680 [==============================] - 118s 18ms/step - loss: 0.3179 - acc: 0.9033 - val_loss: 1.0076 - val_acc: 0.7234
Epoch 6/10
6680/6680 [==============================] - 119s 18ms/step - loss: 0.3156 - acc: 0.9073 - val_loss: 1.0034 - val_acc: 0.7222
Epoch 7/10
6680/6680 [==============================] - 119s 18ms/step - loss: 0.3058 - acc: 0.9121 - val_loss: 0.9969 - val_acc: 0

In [55]:
### Load the model weights with the best validation loss.
model.load_weights('saved_models/weights.best_resnet_augmented.from_scratch.hdf5')

In [56]:
# get index of predicted dog breed for each image in test set
model_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors1]

# report test accuracy
test_accuracy = 100*np.sum(np.array(model_predictions)==np.argmax(test_targets, axis=1))/len(model_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 74.0000%
